<a href="https://colab.research.google.com/github/Athugodage/RuLawSimplification/blob/main/corpus_analysis/text_cleaning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!git clone https://github.com/Athugodage/RuLawSimplification.git

Cloning into 'RuLawSimplification'...
remote: Enumerating objects: 403, done.
remote: Counting objects: 100% (138/138), done.
remote: Compressing objects: 100% (133/133), done.
remote: Total 403 (delta 68), reused 9 (delta 3), pack-reused 265
Receiving objects: 100% (403/403), 207.41 MiB | 13.77 MiB/s, done.
Resolving deltas: 100% (181/181), done.
Updating files: 100% (29/29), done.


In [2]:
!unzip '/content/RuLawSimplification/corpus/the_corpus.zip'

Archive:  /content/RuLawSimplification/corpus/the_corpus.zip
  inflating: the_corpus.csv          


In [3]:
import pandas as pd

df = pd.read_csv('the_corpus.csv')

In [4]:
df.head()

,Название документа,Ссылка,Текст,Комментарий РГ,Date
0,Постановление Конституционного Суда Российской...,https://rg.ru/documents/2022/11/25/document-ob...,Именем Российской Федерации\n Конституционный...,"Бессрочный запрет на профессию для адвокатов, ...",2022-11-25
1,Постановление Пленума Верховного Суда Российск...,https://rg.ru/documents/2022/11/25/document-pl...,В целях обеспечения единства практики применен...,Пленум Верховного суда России утвердил постано...,2022-11-25
2,Федеральный закон от 21 ноября 2022 г. N 455-Ф...,https://rg.ru/documents/2022/11/24/document-po...,Принят Государственной Думой 15 ноября 2022 го...,Единое пособие для беременных женщин и граждан...,2022-11-24
3,Федеральный закон от 21 ноября 2022 г. N 443-Ф...,https://rg.ru/documents/2022/11/24/document-na...,Принят Государственной Думой 10 ноября 2022 го...,На сладкую газировку с 1 июля 2023 года введут...,2022-11-24
4,Федеральный закон от 21 ноября 2022 г. N 448-Ф...,https://rg.ru/documents/2022/11/24/document-by...,Принят Государственной Думой 10 ноября 2022 го...,Приостановленное в этом году бюджетное правило...,2022-11-24


In [5]:
!pip install razdel
from razdel import sentenize

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [40]:
import re
from string import digits



for sent in list(sentenize(df.loc[2017]['Текст'])):
    if re.search('(N \d+, ст. \d{4}; | N \d+ \(.*\))', sent.text) or len(sent.text) < 50:
        pass
    else:
        print(sent.text.lstrip(digits))

Зарегистрирован в Минюсте РФ 30 декабря 2008 г. Регистрационный N 13051  В соответствии с постановлениями Правительства Российской Федерации от 12 августа 1994 г. N 938 "О государственной регистрации автомототранспортных средств и других видов самоходной техники на территории Российской Федерации"1 и от 11 ноября 2005 г. N 679 "О порядке разработки и утверждения административных регламентов исполнения государственных функций (предоставления государственных услуг)"2 приказываю:  1.
Правила регистрации автомототранспортных средств и прицепов к ним в Государственной инспекции безопасности дорожного движения Министерства внутренних дел Российской Федерации3 (приложение N 1).
.2. Административный регламент Министерства внутренних дел Российской Федерации исполнения государственной функции по регистрации автомототранспортных средств и прицепов к ним4 (приложение N 2).
.3. Образец бланка свидетельства о регистрации транспортного средства (приложение N 3).
.4. Образец бланка свидетельства на в

In [53]:
import re
from tqdm import tqdm

tqdm.pandas()

n = 0

def cut_it(text):
    sents = list(sentenize(text))
    start = len(sents) if len(sents) < 40 else 35

    sents = [s.text.lstrip(digits) for s in sents[-start:] if re.match('(\WN \d+\W ст\W \d{4}\W.* | N \d+ \(.*\).*)', sent.text) is None] 
    
    new_text = ''
    for s in sents:
        if len(s) > 50:
            new_text = new_text + ' ' + s
    return new_text
                


df['Cut_original'] = df['Текст'].progress_apply(cut_it)


100%|██████████| 2022/2022 [00:17<00:00, 114.01it/s]


In [42]:
def cut_it(text):
    ## функции отличаются, т.к. в первая режет со второго предложения,
    ## а вторая с первого. Также тут меньше предложений в итоге

    sents = list(sentenize(text))
    end = len(sents) if len(sents) < 30 else 21


    sents = [s.text for s in sents[0:end]] 
    
    new_text = ''
    for s in sents:
        new_text = new_text + ' ' + s
    return new_text
                
df['Cut_comment'] = df['Комментарий РГ'].progress_apply(cut_it)

100%|██████████| 2022/2022 [00:01<00:00, 1610.04it/s]


In [55]:
data = df.copy()

df[['Cut_original', 'Cut_comment']]

,Cut_original,Cut_comment
0,"При этом должны использоваться не чрезмерные,...","Бессрочный запрет на профессию для адвокатов,..."
1,"Суд, установив факт нарушения прав потребител...",Пленум Верховного суда России утвердил постан...
2,"В состав семьи, учитываемый при определении п...",Единое пособие для беременных женщин и гражда...
3,", N 27, ст. 4607) следующие изменения:\n 1) ...",На сладкую газировку с 1 июля 2023 года введу...
4,Перечисление средств по таким контрактам (дог...,Приостановленное в этом году бюджетное правил...
...,...,...
2017,Правила по проведению работ в системе сертифи...,Через десять дней автомобиль можно будет пост...
2018,Правительство Российской Федерации постановля...,Правительство утвердило изменения в правила н...
2019,Правительство Российской Федерации постановля...,"Правительство приняло постановление, расширяю..."
2020,Правительство Российской Федерации постановля...,Специфическим образом теперь должны быть раск...


In [56]:
df[['Cut_original', 'Cut_comment']].to_csv('cut_corpus.csv', index=False)